In [1]:
import syft as sy
import numpy as np
from syft.service.policy.policy import OutputPolicyExecuteOnce, BeachPolicy

In [2]:
node = sy.orchestra.launch(name="test-domain-1", dev_mode=True, reset=True)

Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=9e0cae9404c44393b279932399f7755a in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
root_client = node.login(email="info@openmined.org", password="changethis")

Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [5]:
root_client.register(
    name="Jane Doe",
    email="jane@caltech.edu",
    password="abc123",
    password_verify="abc123",
    institution="Caltech",
    website="https://www.caltech.edu/",
)

SyftSuccess: User 'Jane Doe' successfully registered! To see users, run `[your_client].users`

In [6]:
domain_client = node.login(email="jane@caltech.edu", password="abc123")

Logged into <test-domain-1: High side Domain> as <jane@caltech.edu>


In [7]:
dataset = sy.Dataset(
    name="test",
    asset_list=[
        sy.Asset(
            name="test",
            data=np.array([1, 2, 3]),
            mock=np.array([1, 1, 1]),
            mock_is_real=False,
        )
    ],
)
root_client.upload_dataset(dataset)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.20it/s]


Uploading: test


In [9]:
asset = domain_client.datasets[0].assets[0]

In [10]:
asset

syft.service.dataset.dataset.Asset

In [11]:
# str == str

In [12]:
policy = BeachPolicy(static_arg=asset, prompt=str)

In [13]:
policy.kwarg_rules

{'static_arg': syft.service.policy.policy.Matches,
 'prompt': syft.service.policy.policy.UserOwned}

In [14]:
@sy.syft_function(
    input_policy=policy,
    output_policy=OutputPolicyExecuteOnce()
)
def my_vertex_func(
    static_arg,
    prompt:str
):  
    return prompt + "DEF"
    

SyftSuccess: Syft function 'my_vertex_func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [15]:
request = domain_client.code.request_code_execution(my_vertex_func)

In [20]:
root_client.requests[-1].approve()

Approving request for domain test-domain-1


SyftSuccess: Request 36e03b31ef4740aea17359540e5abcc4 changes applied

In [21]:
res = domain_client.code.my_vertex_func(static_arg=asset, prompt="DEF")

Failed to deserialize custom input policy state. Incorrect argument


In [17]:
res.get()

AttributeError: 'SyftError' object has no attribute 'get'

In [14]:
# def my_vertex_func(
#     ,
#     prompt: str,
#     max_tokens: int,
#     temperature: float,
#     top_p: float,
#     top_k: int,
#     raw_response: bool
# ):
